In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [16]:
def block(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
      """A residual block.
      Arguments:
        x: input tensor.
        filters: integer, filters of the bottleneck layer.
        kernel_size: default 3, kernel size of the bottleneck layer.
        stride: default 1, stride of the first layer.
        conv_shortcut: default True, use convolution shortcut if True,
            otherwise identity shortcut.
        name: string, block label.
      Returns:
        Output tensor for the residual block.
      """
      bn_axis = 3 if keras.backend.image_data_format() == 'channels_last' else 1

      if conv_shortcut:
        shortcut = layers.Conv2D(
            4 * filters, 1, strides=stride, name=name + '_0_conv')(x)
        shortcut = layers.BatchNormalization(
            axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(shortcut)
      else:
        shortcut = x

      x = layers.Conv2D(filters, 1, strides=stride, name=name + '_1_conv')(x)
      x = layers.BatchNormalization(
          axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x)
      x = layers.Activation('relu', name=name + '_1_relu')(x)

      x = layers.Conv2D(
          filters, kernel_size, padding='SAME', name=name + '_2_conv')(x)
      x = layers.BatchNormalization(
          axis=bn_axis, epsilon=1.001e-5, name=name + '_2_bn')(x)
      x = layers.Activation('relu', name=name + '_2_relu')(x)

      x = layers.Conv2D(4 * filters, 1, name=name + '_3_conv')(x)
      x = layers.BatchNormalization(
          axis=bn_axis, epsilon=1.001e-5, name=name + '_3_bn')(x)

      x = layers.Add(name=name + '_add')([shortcut, x])
      x = layers.Activation('relu', name=name + '_out')(x)
      return x

In [17]:
def stack(x, filters, blocks, stride1=2, name=None):
  """A set of stacked residual blocks.
  Arguments:
    x: input tensor.
    filters: integer, filters of the bottleneck layer in a block.
    blocks: integer, blocks in the stacked blocks.
    stride1: default 2, stride of the first layer in the first block.
    name: string, stack label.
  Returns:
    Output tensor for the stacked blocks.
  """
  x = block(x, filters, stride=stride1, name=name + '_block1')
  for i in range(2, blocks + 1):
    x = block(x, filters, conv_shortcut=False, name=name + '_block' + str(i))
  return x

In [18]:
img_inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, 7, strides=2, padding='SAME')(img_inputs)
x = layers.BatchNormalization(axis=1)(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(3, strides=2, padding='SAME')(x)
x = stack(x, 64, 3, stride1=1, name='conv2')
x = stack(x, 128, 4, name='conv3')
x = stack(x, 256, 6, name='conv4')
x = stack(x, 512, 3, name='conv5')
x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
x = layers.Dense(10, activation='softmax', name='predictions')(x)
model = keras.Model(inputs=img_inputs, outputs=x, name="resnet_model")

In [19]:
model.summary()

Model: "resnet_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 16, 16, 64)   9472        input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 16, 16, 64)   64          conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 16, 16, 64)   0           batch_normalization_8[0][0]      
_______________________________________________________________________________________

In [22]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])


Epoch 1/2
242/625 [==========>...................] - ETA: 13:54 - loss: 2.6112 - accuracy: 0.2533  